## Importing the file ###

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
X_train = pd.read_csv("X_train.csv", index_col=[0], header=[0, 1, 2])
X_valid = pd.read_csv("X_valid.csv", index_col=[0], header=[0, 1, 2])
Y_train = pd.read_csv("Y_train.csv", index_col=[0], header=[0])
Y_valid = pd.read_csv("Y_valid.csv", index_col=[0], header=[0])

## Feature Selection 

#### Extract mean columns ####

In [17]:
Drop_columns=[]
for i in range(len(X_train.columns)):
    if X_train.columns[i][1]=='mask' or X_train.columns[i][1]=='time_since_measured':
        Drop_columns.append(X_train.columns[i])

X_train=X_train.drop(Drop_columns,axis=1)

#### Variance thershold ####

In [18]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold()
selector.fit_transform(X_train).shape
#No constant variance columns

(16760, 2496)

#### Dropping high-correlated columns ####

In [19]:
#Define a function for dropping high-coorelated columns
def featureSelectByCorr(df_feat,threshold = 0.9):
    corr = df_feat.corr()
    
    cols = np.full((corr.shape[0],), True, dtype = bool)
    for i in range(corr.shape[0],0,-1):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= threshold:
                if(cols[j]):
                    cols[j] = False
    print("Select features by filtering features with high covariance","(>= "+str(threshold)+")",":")
    print("Original:",df_feat.shape[1],"\tSelected:",np.sum(cols))
    
    return df_feat.iloc[:,cols]




X_corr = featureSelectByCorr(X_train,0.9)





Select features by filtering features with high covariance (>= 0.9) :
Original: 2496 	Selected: 349


## Data scaling or manipulation

#### Normalize all the variable

In [63]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1,1))
X_norm=scaler.fit_transform(X_corr)
X_norm=pd.DataFrame(X_norm)


#### Change Y to numpy array

#### Getting adjusted X_valid according to the work done on X_train

In [64]:
X_valid_corr=X_valid[X_corr.columns]
X_norm_valid=scaler.transform(X_valid_corr)
X_norm_valid=pd.DataFrame(X_norm_valid)

### Please use the data below for further investigation 
#### X_norm = X_train (349 feature)
#### X_norm_valid = X_valid (349 feature)

## Model Construction

In [66]:
from sklearn.metrics import mean_squared_error

#### Random Forest Regressor

In [72]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=30, random_state=0)
regr.fit(X_norm, Y_train)
Y_pred=regr.predict(X_norm_valid)
mean_squared_error(Y_valid, Y_pred, squared=False)


C:\Users\yiuhe\AppData\Local\Temp\ipykernel_6028\3397641522.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_norm, Y_train)


1.832661777925629